In [2]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import *
from datetime import datetime, timezone, timedelta
import json, requests, pandas as pd

StatementMeta(, dd3c7287-0b8e-4a10-a46f-f12ca742d3ae, 3, Finished, Available)

In [3]:
def AuthenticateWithSP():
    key_vault = ""
    tenant = ""
    client = ""
    client_secret = mssparkutils.credentials.getSecret(key_vault , "clientsecret")    
 
    try:
        from azure.identity import ClientSecretCredential
    except Exception:
        !pip install azure.identity
        from azure.identity import ClientSecretCredential
 
    # Generates the access token for the Service Principal
    api = 'https://analysis.windows.net/powerbi/api/.default'
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
    access_token = auth.get_token(api)
    access_token = access_token.token
    header = {'Authorization': f'Bearer {access_token}'}
    print('\nSuccessfully authenticated.')
    return header


StatementMeta(, dd3c7287-0b8e-4a10-a46f-f12ca742d3ae, 4, Finished, Available)

In [4]:
header= AuthenticateWithSP()

StatementMeta(, dd3c7287-0b8e-4a10-a46f-f12ca742d3ae, 5, Finished, Available)


Successfully authenticated.


In [5]:
#########################################################################################
# Variable definition
#########################################################################################

base_url = 'https://api.powerbi.com/v1.0/myorg/'
startDate= datetime.utcnow() + timedelta(days=-30) 
startDate = startDate.replace(hour=0, minute=00,second=00)
startDate=startDate.strftime("%Y-%m-%dT%H:%M:%S.000Z")

WhileCondition=-30
firstrun=True

StatementMeta(, dd3c7287-0b8e-4a10-a46f-f12ca742d3ae, 6, Finished, Available)

In [6]:
while WhileCondition < 0:

    EndtDate= datetime.utcnow() + timedelta(days=WhileCondition) 
    EndtDate = EndtDate.replace(hour=23, minute=59,second=59)
    EndtDate=EndtDate.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    print(startDate)

    
    API_url = base_url+"admin/activityevents?startDateTime='"+startDate+"'&endDateTime='"+EndtDate+"'"
    response = requests.get(API_url, headers=header)
    
    if response.status_code == 429:
        print("sleep")
        time.sleep(3601)
        header= AuthenticateWithSP()
        response = requests.get(API_url, headers=header)
     
    JSONContent = json.loads(response.content)
    JSONContentExplode = JSONContent['activityEventEntities'] 
    lastResultSet = JSONContent['lastResultSet']
    
    while lastResultSet==False :
        continuationUri = JSONContent['continuationUri']
        response = requests.get(continuationUri, headers=header)
        if response.status_code == 429:
            print("sleep")
            time.sleep(3601)
            header= AuthenticateWithSP()
            response = requests.get(continuationUri, headers=header)

        JSONContent = json.loads(response.content)
        JSONContentExplode=JSONContentExplode+JSONContent['activityEventEntities']
        lastResultSet = JSONContent['lastResultSet']


    if len(JSONContentExplode)>0:
                if firstrun is True  :
                        DailyActivityLogs=spark.read.json(sc.parallelize([json.dumps(JSONContentExplode)]))
                        DailyActivityLogs= DailyActivityLogs.withColumn("CreationTime",to_timestamp(DailyActivityLogs.CreationTime, "yyyy-MM-dd'T'HH:mm:ss") )
                        DailyActivityLogs=DailyActivityLogs.withColumn("PartitionDate",to_date( col("CreationTime")))
                        firstrun=False
                else:
                        temp=spark.read.json(sc.parallelize([json.dumps(JSONContentExplode)]))
                        temp= temp.withColumn("CreationTime",to_timestamp(temp.CreationTime, "yyyy-MM-dd'T'HH:mm:ss") )
                        temp=temp.withColumn("PartitionDate",to_date( col("CreationTime")))
                        DailyActivityLogs=DailyActivityLogs.unionByName(temp, allowMissingColumns=True)

                

    WhileCondition= WhileCondition+1
    startDate= datetime.utcnow() + timedelta(days=WhileCondition) 
    startDate = startDate.replace(hour=0, minute=00,second=00)
    startDate=startDate.strftime("%Y-%m-%dT%H:%M:%S.000Z")

StatementMeta(, dd3c7287-0b8e-4a10-a46f-f12ca742d3ae, 7, Finished, Available)

2023-10-30T00:00:00.000Z
2023-10-31T00:00:00.000Z
2023-11-01T00:00:00.000Z
2023-11-02T00:00:00.000Z
2023-11-03T00:00:00.000Z
2023-11-04T00:00:00.000Z
2023-11-05T00:00:00.000Z
2023-11-06T00:00:00.000Z
2023-11-07T00:00:00.000Z
2023-11-08T00:00:00.000Z
2023-11-09T00:00:00.000Z
2023-11-10T00:00:00.000Z
2023-11-11T00:00:00.000Z
2023-11-12T00:00:00.000Z
2023-11-13T00:00:00.000Z
2023-11-14T00:00:00.000Z
2023-11-15T00:00:00.000Z
2023-11-16T00:00:00.000Z
2023-11-17T00:00:00.000Z
2023-11-18T00:00:00.000Z
2023-11-19T00:00:00.000Z
2023-11-20T00:00:00.000Z
2023-11-21T00:00:00.000Z
2023-11-22T00:00:00.000Z
2023-11-23T00:00:00.000Z
2023-11-24T00:00:00.000Z
2023-11-25T00:00:00.000Z
2023-11-26T00:00:00.000Z
2023-11-27T00:00:00.000Z
2023-11-28T00:00:00.000Z


In [7]:
spark.conf.set( "spark.sql.sources.partitionOverwriteMode", "dynamic")

DailyActivityLogs.write.format('delta')\
.mode("overwrite")\
.option("mergeSchema", "true")\
.partitionBy("PartitionDate")\
.save("Files/EventLogs")

StatementMeta(, dd3c7287-0b8e-4a10-a46f-f12ca742d3ae, 8, Finished, Available)